# Retrieval-Augmented Generation (RAG) Demo

This notebook demonstrates a simple retrieval-augmented generation workflow using a custom search engine.
I practiced using Github Copilot to write the code cell by cell. 

In [ ]:
#Download the custom search engine and documents json file from the Datatalks Club GitHub repository.
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

--2025-05-28 15:17:07--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.007s  

2025-05-28 15:17:08 (84.5 MB/s) - ‘documents.json’ saved [658332/658332]



In [28]:
#Import necessary libraries
import json
import minsearch
from openai import OpenAI

In [29]:
# Load the documents from the JSON file
with open('documents.json', 'r') as f:
    docs_raw = json.load(f)

In [30]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [31]:
# Create the search index
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [32]:
# Set the query 
q = 'the course has already started, can I still enroll?'

In [33]:
# Fit the index with the documents
index.fit(documents)

In [34]:
# Set boost values to give more importance to certain fields during search
boost = {
    "question": 3,
    "section": 0.5
}

In [35]:
# Search the index with the query, filtering by course and applying boost
results = index.search(
    q,
    num_results=5,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost
)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [36]:
# Create an OpenAI client instance
client = OpenAI()

In [ ]:
# Send a request with our simple query to the OpenAI model
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": q},
    ]
)

NameError: name 'response' is not defined

In [ ]:
# Print the response from the OpenAI model
print(response.choices[0].message.content)

In [24]:
# Create prompt template for the question and context
prompt_template = """
You are a course assistant for the Data Engineering Zoomcamp. 
You will be given a question. Your task is to answer the question based on the CONTEXT from the FAQ Documents. Use only facts from the CONTEXT to answer the question. If you don't know the answer, say "I don't know".
Question: {question}
Context: {context}
Answer: 
""".strip()

In [ ]:
# Create context of documents from our search results
context = ""

for doc in results: 
    context += f"Question: {doc['question']}\n"
    context += f"Answer: {doc['text']}\n"
    context += f"Section: {doc['section']}\n\n"

In [ ]:
# Create the prompt
prompt = prompt_template.format(
    question=q,
    context=context
).strip()
# Print the prompt
print(prompt)

In [38]:
# Send the prompt with context  to the OpenAI model and get the response
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt},
    ]
)


In [ ]:
# Print the response with our FAQ context 
print(response.choices[0].message.content)

Yes, you can still enroll in the course after it has started. Even if you don't register, you're eligible to submit the homework. However, be aware that there will be deadlines for turning in the final projects, so it's important not to leave everything for the last minute.
